<p style='text-align: right;'> <b>Stefano Caglio</b> © 2020<br>
📫 <a href="mailto:stefano.caglio@gmail.com">stefano.caglio@gmail.com</a><br>
🔗 <a href="https://www.stefanocaglio.com">stefanocaglio.com</a></p>

# Automatic email sender - v.1.0.0 - 2020/11/27

### Specifications

What this automatic email sender does:
- Import data (email address + other fields for customizing email) from an external csv
- Search and attach customized files from an external folder
- Access and send emails using an SMTP server

### Table of content
* [Import modules](#chapter-1)
* [Logger parameters adjustment](#chapter-2)
* [Use pandas for importing csv data file and managing it](#chapter-3)
* [Main loops](#chapter-4)
* [Switch off logging](#chapter-5)

#### Import modules <a id="chapter-1"></a>

In [1]:
# openpyxl per lavorare con fogli Excel
# import openpyxl, datetime
# from datetime import date
import smtplib, os
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
import pandas as pd
import logging
import glob

#### Logger parameters adjustment <a id="chapter-2"></a>

In [2]:
logger = logging.getLogger('my_logger')

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s %(levelname)-8s %(message)s',
    filename='app1.log', # write to this file
    filemode='a+' # open in append mode
    )

# logging.info('This will get logged to a file') # Just for log file testing porpouse

#### Use pandas for importing csv data file and managing it <a id="chapter-3"></a>

In [31]:
# Choose the appropriate CSV file
# file = r'C:/Users/Lenovo/Documents/Python Scripts/Licei_scelti_9COL.csv'
file = r'C:/Users/Lenovo/Desktop/MAD/MAD_Latex/Licei_scelti_9COL_TEST.csv'
dfmail = pd.read_csv(file, sep=';')
dfmail.rename(columns = {'Indirizzo PEC Autonomia':'PEC'}, inplace = True)
# Have a look @ old and new formatting at https://pyformat.info/
logging.info('Load data from {}'.format(file))

In [33]:
# TO be used for data sanity check
dfmail.head()
dfmail.tail()

,Provincia,Codice,Tipologia,Denominazione,Indirizzo,Civico,Comune,CAP,PEC
0,BG,BGPS016011,Istituto Istruzione Secondario Secondo grado,L. Scientifico - Valle Seriana,Via Marconi,45,GAZZANIGA,24025,stefano.caglio@gmail.com
1,BG,BGPS03401E-01,Istituto Istruzione Secondario Secondo grado,L. Scientifico - Don Milani,Via Rubini,2,ROMANO DI LOMBARDIA,24058,stefano.caglio@troteclaser.com


#### Main loops <a id="chapter-4"></a>
- 1st loop: index representing row# of the data
- 2nd loop: search path of files to be attached 

In [21]:
logging.info('START SENDING EMAIL')

for i in dfmail.index:
    for name in glob.glob('C:/Users/Lenovo/Desktop/MAD/MAD_Latex/*'+ dfmail.Codice[i] +'*.pdf'):
        # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        # * * * * * * *    WRITE HERE THE HTML EMAIL  * * * * * * * *
        # Mail con allegato il pdf MAD preparata in LaTeX
        TO = dfmail.PEC[i]
        ##CC = 'stefano.caglio@gmail.com'  # Uncomment when in production to receive a copy of each email
        CC = ''
        SUBJECT = 'Invio modulo di messa a disposizione a {}, codice {}'.format(dfmail.Denominazione[i], dfmail.Codice[i])
        
        # exchange Sign In
        exchange_sender = 'info@pec.stefanocaglio.com'
        exchange_passwd = 'Trodatit2000'

        # instance of MIMEMultipart 
        msg = MIMEMultipart('alternative')

        # storing the senders email address   
        msg['From'] = exchange_sender

        # storing the receivers email address  
        msg['To'] = TO
        msg['Cc'] = CC

        # storing the subject  
        msg['Subject'] = SUBJECT

        
        # Create the body of the message (a plain-text and an HTML version).
        text = "Gentile Dirigente Scolastico,\n\nmi auguro di trovare ben Lei e tutti i Suoi collaboratori anche se in una situazione storica che, fortunatamente, viviamo per la prima volta.\n\nLe scrivo per sottoporLe la mia messa a disposizione, che trova in allegato a questa mail.\n\nNel caso La considerasse in linea con i Vostri requisiti, trova tutti i miei riferimenti sia sul modulo che in calce a questa email.\n\nCon i più cordiali saluti\n\n\n     Stefano Caglio\n\nTelefono: +39 333 3757003  / / /  +39 345 0457911\nMail: stefano.caglio@gmail.com  / / /  info@pec.stefanocaglio.com\nsito stefanocaglio.com"

        html = """\
        <html>
          <head></head>
          <body>
            <p>Gentile Dirigente Scolastico,<br><br>
                mi auguro di trovare ben Lei e tutti i Suoi collaboratori anche se in una situazione storica che, fortunatamente, viviamo per la prima volta.<br><br>
                Le scrivo per sottoporLe la mia messa a disposizione, che trova in allegato a questa mail.<br><br>
                Nel caso La considerasse in linea con i Vostri requisiti, trova tutti i miei riferimenti sia sul modulo che in calce a questa email.<br><br>
                Con i più cordiali saluti<br><br><br>

                     Stefano Caglio<br><br>
                📱 +39 333 3757003  / / /  +39 345 0457911<br>
                📫 <a href="mailto:stefano.caglio@gmail.com">stefano.caglio@gmail.com</a>  / / /  <a href="mailto:info@pec.stefanocaglio.com">info@pec.stefanocaglio.com</a><br>
                🔗 <a href="https://www.stefanocaglio.com">stefanocaglio.com</a>
            </p>
          </body>
        </html>
        """

        # Record the MIME types of both parts - text/plain and text/html.
        part1 = MIMEText(text, 'plain')
        part2 = MIMEText(html, 'html')

        # Attach parts into message container.
        # According to RFC 2046, the last part of a multipart message, in this case
        # the HTML message, is best and preferred.
        msg.attach(part1)
        msg.attach(part2)

        # Open the file to be sent  
        attachment = open(name, 'rb')

        # Instance of MIMEBase and named as p 
        p = MIMEBase('application', 'octet-stream')

        # To change the payload into encoded form 
        p.set_payload((attachment).read()) 

        # Encode into base64 
        encoders.encode_base64(p) 

        nomePDF = 'MAD_StefanoCaglio_{}.pdf'.format(dfmail.Codice[i])
        p.add_header('Content-Disposition', 'attachment; filename= %s' % nomePDF) 

        # Attach the instance 'p' to instance 'msg' 
        msg.attach(p)
        
        # Login to the SMTP server
        server = smtplib.SMTP_SSL('smtp.pec-email.com', 465)
        server.ehlo()
        #server.starttls()
        server.login(exchange_sender, exchange_passwd)

        # Converts the Multipart msg into a string 
        mailtext = msg.as_string() 

        try:
            server.sendmail(exchange_sender, msg["To"].split(",") + msg["Cc"].split(","), mailtext)
            logging.info('Mail sent to {} attaching {}'.format(TO,nomePDF))
        except:
            logging.warning('ERROR sending mail to {}'.format(TO))

        server.quit()
        
logging.info('END SENDING EMAIL')

###### ALWAYS SWITCH OFF LOGGING!!! <a id="chapter-5"></a>

In [39]:
logging.shutdown()   # to be placed at the end of the script